In [1]:
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense
from sklearn.utils import shuffle

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()
mean = np.mean(train_data,axis=0)
train_data -= mean
std = np.std(train_data, axis=0)
train_data /= std

test_data -= mean
test_data /= std

57344/57026 [==============================] - 0s 6us/step


In [0]:
def build_model():
  model = Sequential()
  model.add(Dense(64,activation="relu",input_shape=(train_data.shape[1],)))
  model.add(Dense(64,activation="relu"))
  model.add(Dense(1))
  model.compile(loss="mse",optimizer="rmsprop",metrics=["mae"])
  return model

In [0]:
k = 4
fold_sample = len(train_data)//k
all_scores = []

In [9]:
#using iterated cross validation with shuffling since we have few data
for n in range(k):
  print("Running k-fold ", n," times")
  train_data, train_labels = shuffle(train_data, train_labels)
  mae_scores = []
  for i in range(k):
    print("Processing fold ",i)
    val_data = train_data[i*fold_sample:(i+1)*fold_sample]
    val_target = train_labels[i*fold_sample:(i+1)*fold_sample]
    x_data = np.concatenate([train_data[:i*fold_sample],train_data[(i+1)*fold_sample:]],axis=0)
    x_label = np.concatenate([train_labels[:i*fold_sample],train_labels[(i+1)*fold_sample:]],axis=0)
    model = build_model()
    model.fit(x_data,x_label,epochs=100,verbose=0,batch_size=12)
    val_mse, val_mae = model.evaluate(val_data,val_target,verbose=0)
    mae_scores.append(val_mae)
  all_scores.append(np.mean(mae_scores))

Running k-fold  0  times
Processing fold  0
Processing fold  1
Processing fold  2
Processing fold  3
Running k-fold  1  times
Processing fold  0
Processing fold  1
Processing fold  2
Processing fold  3
Running k-fold  2  times
Processing fold  0
Processing fold  1
Processing fold  2
Processing fold  3
Running k-fold  3  times
Processing fold  0
Processing fold  1
Processing fold  2
Processing fold  3


In [10]:
np.mean(all_scores)

2.3647605050539617

In [0]:
#computing the average of the per-epoch MAE scores for all folds 
average_mae_scores = np.mean(mae_scores[:4],axis=0)

In [23]:
#after fine tuning the hyper-parameters we then run the model on the entire dataset
model = build_model()
model.fit(train_data,train_labels,epochs=80,verbose=0,batch_size=16)
mse, mae = model.evaluate(test_data,test_labels)

102/102 [==============================] - 1s 5ms/step


In [25]:
print("Mean absolute error : ",mae)

Mean absolute error :  2.8706756386102414
